In [2]:
import matplotlib.pyplot as plt
import tensorflow as tf
import cv2
import numpy as np
content_image = None

In [3]:
STYLE_IMAGE_NAME = 'IMAGE_7' #@param ['IMAGE_1', 'IMAGE_2', 'IMAGE_3', 'IMAGE_4', 'IMAGE_5', 'IMAGE_6', 'IMAGE_7', 'IMAGE_6', 'IMAGE_8', 'IMAGE_9', 'IMAGE_10', 'IMAGE_11', 'IMAGE_12', 'IMAGE_13', 'IMAGE_14']

corresponding_url = {
  'IMAGE_1':'https://upload.wikimedia.org/wikipedia/commons/0/0a/The_Great_Wave_off_Kanagawa.jpg',
  'IMAGE_2':'https://upload.wikimedia.org/wikipedia/commons/b/b4/Vassily_Kandinsky%2C_1913_-_Composition_7.jpg',
  'IMAGE_3':'https://upload.wikimedia.org/wikipedia/commons/6/68/Pillars_of_creation_2014_HST_WFC3-UVIS_full-res_denoised.jpg',
  'IMAGE_4':'https://upload.wikimedia.org/wikipedia/commons/thumb/e/ea/Van_Gogh_-_Starry_Night_-_Google_Art_Project.jpg/1024px-Van_Gogh_-_Starry_Night_-_Google_Art_Project.jpg',
  'IMAGE_5':'https://upload.wikimedia.org/wikipedia/commons/b/b7/JMW_Turner_-_Nantes_from_the_Ile_Feydeau.jpg',
  'IMAGE_6':'https://upload.wikimedia.org/wikipedia/commons/c/c5/Edvard_Munch%2C_1893%2C_The_Scream%2C_oil%2C_tempera_and_pastel_on_cardboard%2C_91_x_73_cm%2C_National_Gallery_of_Norway.jpg',
  'IMAGE_7':'https://upload.wikimedia.org/wikipedia/en/4/4c/Les_Demoiselles_d%27Avignon.jpg',
  'IMAGE_8':'https://upload.wikimedia.org/wikipedia/en/3/3c/Pablo_Picasso%2C_1911-12%2C_Violon_%28Violin%29%2C_oil_on_canvas%2C_Kr%C3%B6ller-M%C3%BCller_Museum%2C_Otterlo%2C_Netherlands.jpg',
  'IMAGE_9':'https://upload.wikimedia.org/wikipedia/en/7/7f/Pablo_Picasso%2C_1911%2C_Still_Life_with_a_Bottle_of_Rum%2C_oil_on_canvas%2C_61.3_x_50.5_cm%2C_Metropolitan_Museum_of_Art%2C_New_York.jpg',
  'IMAGE_10':'https://upload.wikimedia.org/wikipedia/commons/3/36/Large_bonfire.jpg',
  'IMAGE_11':'https://upload.wikimedia.org/wikipedia/commons/0/0d/Derkovits_Gyula_Woman_head_1922.jpg',
  'IMAGE_12':'https://upload.wikimedia.org/wikipedia/commons/8/8e/Untitled_%28Still_life%29_%281913%29_-_Amadeo_Souza-Cardoso_%281887-1918%29_%2817385824283%29.jpg',
  'IMAGE_13':'https://upload.wikimedia.org/wikipedia/commons/3/37/Derkovits_Gyula_Talig%C3%A1s_1920.jpg',
  'IMAGE_14':'https://upload.wikimedia.org/wikipedia/commons/7/7d/Amadeo_de_Souza-Cardoso%2C_1915_-_Landscape_with_black_figure.jpg'
}

style_image_path = tf.keras.utils.get_file(STYLE_IMAGE_NAME + ".jpg", corresponding_url[STYLE_IMAGE_NAME])

In [ ]:
# Creates upload (for uploading images) and preview (for plotting the image)
import ipywidgets
from IPython.display import display
import ipywidgets as widgets
from PIL import Image
import numpy as np
import io

def button_click(change):
    global content_image
    print('\n')
    print(dir(uploader))
    img = Image.open(io.BytesIO(uploader.data[-1]))
    content_image = img
    # display(img)
    
uploader = widgets.FileUpload()
show_button = widgets.Button(description='Preview')
show_button.on_click(button_click)

widgets.VBox([widgets.Label(), uploader, show_button])

In [ ]:
# Opens the webcam so that we can click on the frame on which the style will be applied

from IPython.display import HTML, Audio
from google.colab.output import eval_js
from base64 import b64decode
import numpy as np
import io
from PIL import Image

VIDEO_HTML = """
<video autoplay
 width=%d height=%d style='cursor: pointer;'></video>
<script>

var video = document.querySelector('video')

navigator.mediaDevices.getUserMedia({ video: true })
  .then(stream=> video.srcObject = stream)
  
var data = new Promise(resolve=>{
  video.onclick = ()=>{
    var canvas = document.createElement('canvas')
    var [w,h] = [video.offsetWidth, video.offsetHeight]
    canvas.width = w
    canvas.height = h
    canvas.getContext('2d')
          .drawImage(video, 0, 0, w, h)
    video.srcObject.getVideoTracks()[0].stop()
    video.replaceWith(canvas)
    resolve(canvas.toDataURL('image/jpeg', %f))
  }
})
</script>
"""
def take_photo(filename='photo.jpg', quality=0, size=(800,600)):
    global content_image
    display(HTML(VIDEO_HTML % (size[0],size[1],quality)))
    data = eval_js("data")
    binary = b64decode(data.split(',')[1])
    f = io.BytesIO(binary)
    content_image = Image.open(f)
    print('\nImage captured! 🤳')

take_photo()

In [8]:
def load_content_img(image):
  img = tf.convert_to_tensor(image)
  img = tf.image.convert_image_dtype(img, tf.float32)
  print(image.shape)
  img = img[tf.newaxis, :]
  return img  

In [9]:
def load_style_img(image):
  img = tf.io.read_file(image)
  img = tf.io.decode_image(img, channels=3)
  img = tf.image.convert_image_dtype(img, tf.float32)
  img = img[tf.newaxis, :]
  return img

In [10]:
def preprocess_image(image, target_dim):
  print(image.shape) # (1, 543, 966, 3)
  shape = tf.cast(tf.shape(image)[1:-1], tf.float32)
  print(shape)
  short_dim = min(shape)
  scale = target_dim / short_dim
  print(scale)
  new_shape = tf.cast(shape * scale, tf.int32)
  print(new_shape)
  image = tf.image.resize(image, new_shape)
  print(image.shape)
  image = tf.image.resize_with_crop_or_pad(image, target_dim, target_dim)
  print(image.shape)
  return image

# (1, 543, 966, 3)
# tf.Tensor([543. 966.], shape=(2,), dtype=float32)
# tf.Tensor(0.70718235, shape=(), dtype=float32)
# tf.Tensor([384 683], shape=(2,), dtype=int32)
# (1, 384, 683, 3)
# (1, 384, 384, 3)  

In [11]:
def imshow(image, title=None):
  if len(image.shape) > 3:
    image = tf.squeeze(image, axis=0)

  plt.imshow(image)
  if title:
    plt.title(title)

In [ ]:
# print(content_image)
# print("--------------------------------------------")
content_image = np.array(content_image)
# print(content_image)
content_image = load_content_img(content_image)
preprocessed_content_image = preprocess_image(content_image, 384)

In [14]:
print('Preprocessing the content image...')
print('Content image shape:', preprocessed_content_image.shape)

Preprocessing the content images...
Content image shape: (1, 384, 384, 3)


In [ ]:
model_type = "int8" #@param ["int8", "float16"]
source_image = preprocessed_content_image
if model_type == "float16":
    preprocessed_source_image = preprocess_image(source_image, target_dim=224) 
else:
    preprocessed_source_image = preprocess_image(source_image, target_dim=512) 
preprocessed_source_image.shape

In [16]:
model_dict = {
    "float16": "lite-model_cartoongan_fp16_1.tflite",
    "int8": "lite-model_cartoongan_int8_1.tflite"
}

interpreter = tf.lite.Interpreter(model_path=model_dict[model_type])
input_details = interpreter.get_input_details()


interpreter.allocate_tensors()
interpreter.set_tensor(input_details[0]['index'], preprocessed_source_image)
interpreter.invoke()

raw_prediction = interpreter.tensor(
    interpreter.get_output_details()[0]['index'])()

In [17]:
output = (np.squeeze(raw_prediction)+1.0)*127.5
output = np.clip(output, 0, 255).astype(np.uint8)
output = cv2.cvtColor(output, cv2.COLOR_BGR2RGB)

In [ ]:
plt.figure(figsize=(20, 20))
plt.subplot(1, 2, 1)
plt.imshow(preprocessed_content_image[0])
plt.title('Source image')
plt.subplot(1, 2, 2)
plt.imshow(output)
plt.title('Cartoonized image')
plt.show()